 1MG- Data Analyst Test
 
 Họ tên: Đặng Thị Mỹ Linh

In [1]:
import pandas as pd, numpy as np
import os

from bokeh.plotting import figure,output_file, save
from bokeh.io import output_notebook, show, export_png
from bokeh.layouts import column, row
from bokeh.models import (ColumnDataSource, HoverTool,Range1d,LinearAxis,
NumeralTickFormatter,NumberFormatter,DataTable,DatetimeTickFormatter,
Legend, LegendItem, FactorRange)
from bokeh.transform import factor_cmap
output_notebook()


Loading BokehJS ...

In [2]:
customers = pd.read_csv('./olist_customers_dataset.csv')
geolocation = pd.read_csv('./olist_geolocation_dataset.csv')
order_items = pd.read_csv('./olist_order_items_dataset.csv')
payments = pd.read_csv('./olist_order_payments_dataset.csv')
reviews = pd.read_csv('./olist_order_reviews_dataset.csv')
orders = pd.read_csv('./olist_orders_dataset.csv')
products = pd.read_csv('./olist_products_dataset.csv')
sellers = pd.read_csv('./olist_sellers_dataset.csv')
category = pd.read_csv('./product_category_name_translation.csv')

In [3]:
# Function để summary
def describe_data(df, name):
    print('Name of data set: {}'.format(name))
    print('Dataset Shape: {}'.format(df.shape))
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index().rename(columns={'index':'Name'})
    summary['Missing'] = df.isnull().sum().values
    summary['Uniques'] = df.nunique().values
    summary['Count_value'] = df.count().values
    try:
        statis = df.describe().transpose().reset_index().rename(
            columns={'index':'Name'})[['Name','mean','std','25%','50%','75%','max']]
        summary = summary.merge(statis,on = 'Name', how = 'left')
    except:
        pass
    value_example = df.head(2).transpose().reset_index().rename(columns={'index':'Name',0:'value1',1:'value2'})
    summary = summary.merge(value_example,on = 'Name', how = 'left')
    summary.index = np.arange(1,len(summary)+1)
    return summary

# Function check mối quan hệ giữa các biến trong cùng 1 bảng dữ liệu
def check_references_relationship(df,var1,var2):
    count_var2_by_var1 = df.groupby([var1]).agg({var2:pd.Series.nunique})
    if len(count_var2_by_var1.loc[count_var2_by_var1[var2]>1])>0:
        message = '1 giá trị của {var1} có thể có nhiều giá trị {var2} - quan hệ 1-nhiều'.format(var1=var1,var2=var2)
    else:
        message = '1 giá trị của {var1} chỉ tương ứng 1 giá trị {var2} - quan hệ 1-1'.format(var1=var1,var2=var2)
    print(message)

# Function plot distribution cho 1 demension
def plot_bar_distribution_1d(df,demension,measure,title,y_label,x_label,fill_color,sort=True,count=True):
    if count== True:
        df_group = df.groupby([demension]).agg({measure:pd.Series.nunique})
        df_group['pct'] = df_group[measure]/df_group[measure].sum()
    else:
        df.set_index([demension],inplace=True,drop=True)
        df_group = df
    
    x_range = df.groupby([demension])
    sorted_factors = df_group.sort_values([measure],ascending=False).index.tolist()
    source_df_group = ColumnDataSource(df_group)
    p1 = figure(plot_width = 950,
                plot_height=300,
                title = title,
                x_range=x_range,
                toolbar_location=None,
          )
    p1.vbar(x=demension,top = measure,
          width=0.4,source=source_df_group,line_color="silver",fill_color=fill_color,alpha=0.8
            
          )
    if count== True:
        p1.text(df_group.index.values.tolist(),
               (df_group[measure]
                +df_group[measure].max()*0.2).values.tolist(),
               text=df_group[measure].apply(lambda x: "{:,.0f}".format(x)).values.tolist(), 
               alpha=2,text_font_style ='normal',
               text_font_size="7pt", 
               text_baseline="middle",
               text_align="center")
        p1.text(df_group.index.values.tolist(),
               (df_group[measure]
                +df_group[measure].max()*0.1).values.tolist(),
               text=df_group['pct'].apply(lambda x: "{:,.1f} %".format(x*100)).values.tolist(), 
               alpha=2,text_font_style ='normal',
               text_font_size="7pt", 
               text_baseline="middle",
               text_align="center",
               text_color ='red')
    else:
        p1.text(df_group.index.values.tolist(),
               (df_group[measure]
                 +df_group[measure].max()*0.1).values.tolist(),
               text=df_group[measure].apply(lambda x: "{:,.1f}".format(x)).values.tolist(), 
               alpha=2,text_font_style ='normal',
               text_font_size="7pt", 
               text_baseline="middle",
               text_align="center")
    if sort == True:
        p1.x_range.factors = sorted_factors
    else:
        pass
    p1.axis.axis_line_color = 'silver'
    p1.axis.axis_label_text_font_style = 'bold'
    p1.yaxis.axis_label = y_label
    p1.xaxis.axis_label = x_label
    p1.xaxis.major_label_orientation = 1.57
    p1.xgrid.grid_line_color=None
    p1.xaxis.major_tick_in=0
    p1.y_range.start=0.1
    p1.x_range.group_padding = 3
    p1.x_range.range_padding=0.01
    p1.yaxis.formatter=NumeralTickFormatter(format="0,0")
    return p1

def plot_bar_distribution_2d(df,demension1,demension2,measure,title,y_label,x_label,sort=True, pct=True, text=True):
    df_group = df.set_index([demension1,demension2])
    x_range = df.groupby([demension1,demension2])
    sorted_factors = df_group.sort_values([measure],ascending=False).index.tolist()
    source_df_group = ColumnDataSource(df_group)
    p1 = figure(plot_width = 950,
                plot_height=400,
                title = title,
                x_range=x_range,
                toolbar_location=None,
          )
    p1.vbar(x=demension1+'_'+demension2,top = measure,
          width=0.4,source=source_df_group,line_color="silver",fill_color='color'
            
          )
    if text==True and pct==True:
        p1.text(df_group.index.values.tolist(),
               (df_group[measure]
                +df_group[measure].max()*0.2).values.tolist(),
               text=df_group[measure].apply(lambda x: "{:,.1f} %".format(x*100)).values.tolist(), 
               alpha=2,text_font_style ='normal',
               text_font_size="6pt", 
               text_baseline="middle",
               text_align="center")
        p1.yaxis.formatter=NumeralTickFormatter(format="0,0 %")
    elif text==True and pct==False:
    
        p1.text(df_group.index.values.tolist(),
               (df_group[measure]
                +df_group[measure].max()*0.2).values.tolist(),
               text=df_group[measure].apply(lambda x: "{:,.0f} ".format(x)).values.tolist(), 
               alpha=2,text_font_style ='normal',
               text_font_size="6pt", 
               text_baseline="middle",
               text_align="center")
        p1.yaxis.formatter=NumeralTickFormatter(format="0,0")
        
    else:
        p1.text(df_group.index.values.tolist(),
               (df_group[measure]*1.1
                +df_group['distance']).values.tolist(),
               text=df_group['text'].apply(lambda x: "+ {:,.0f} %".format(x*100) if x >0 else  (''if x==0 else "- {:,.0f} %".format(x*100))).values.tolist(), 
               alpha=2,text_font_style ='normal',
               text_font_size="7pt", 
               text_baseline="middle",
               text_align="center")
        p1.yaxis.formatter=NumeralTickFormatter(format="0,0")
        
    
    if sort == True:
        p1.x_range.factors = sorted_factors
    else:
        pass
    p1.axis.axis_line_color = 'silver'
    p1.axis.axis_label_text_font_style = 'bold'
    p1.yaxis.axis_label = y_label
    p1.xaxis.axis_label = x_label
    p1.xaxis.major_label_orientation = 1.57
    p1.xgrid.grid_line_color=None
    p1.xaxis.major_tick_in=0
    p1.y_range.start=0
    p1.x_range.group_padding = 3
    p1.x_range.range_padding=0.02
    
    return p1

def format_fl_to_int(df):
    for column in df.columns:
        if df[column].dtypes =='float64':
            df[column] = df[column].apply(lambda x: "{:,.0f}".format(x))

### Tập dữ liệu customers

In [4]:
describe_data(customers,'customers')

Name of data set: customers
Dataset Shape: (99441, 5)


,Name,dtypes,Missing,Uniques,Count_value,mean,std,25%,50%,75%,max,value1,value2
1,customer_id,object,0,99441,99441,NaN,NaN,NaN,NaN,NaN,NaN,06b8999e2fba1a1fbc88172c00ba8bc7,18955e83d337fd6b2def6b18a428ac77
2,customer_unique_id,object,0,96096,99441,NaN,NaN,NaN,NaN,NaN,NaN,861eff4711a542e4b93843c6dd7febb0,290c77bc529b7ac935b93aa66c333dc3
3,customer_zip_code_prefix,int64,0,14994,99441,35137.474583,29797.938996,11347.0,24416.0,58900.0,99990.0,14409,9790
4,customer_city,object,0,4119,99441,NaN,NaN,NaN,NaN,NaN,NaN,franca,sao bernardo do campo
5,customer_state,object,0,27,99441,NaN,NaN,NaN,NaN,NaN,NaN,SP,SP


Tập dữ liệu customers gồm 99441 dòng với 5 cột có đặc điểm:
- Các cột đều không có dữ liệu bị thiếu
- 4/5 cột dữ liệu thuộc dang object (string)

Nếu xem tập dữ liệu customer là 1 bảng thì có thể nhận thấy:
- customer_id có thể là primary key (do tính not null, uniques value = count row)
- customer_unique_id có thể là foreign key (refernces tới các bảng liên quan) 
- customer_zip_code_prefix có thể lầ foreign key (refernces tới parent table là geolocation )
- customer_city có thể lầ foreign key (refernces tới parent table là geolocation )
- customer_state có thể lầ foreign key (refernces tới parent table là geolocation )

** Việc xác định foreign key nhằm xác định relation giữa các bảng dữ liệu để có thể mapping các bảng dữ liệu với nhau

In [5]:
for var in ('customer_id','customer_zip_code_prefix','customer_city','customer_state'):
    check_references_relationship(customers,'customer_unique_id',var)

1 giá trị của customer_unique_id có thể có nhiều giá trị customer_id - quan hệ 1-nhiều
1 giá trị của customer_unique_id có thể có nhiều giá trị customer_zip_code_prefix - quan hệ 1-nhiều
1 giá trị của customer_unique_id có thể có nhiều giá trị customer_city - quan hệ 1-nhiều
1 giá trị của customer_unique_id có thể có nhiều giá trị customer_state - quan hệ 1-nhiều


In [6]:
for var in ('customer_unique_id','customer_zip_code_prefix','customer_city','customer_state'):
    check_references_relationship(customers,'customer_id',var)
    

1 giá trị của customer_id chỉ tương ứng 1 giá trị customer_unique_id - quan hệ 1-1
1 giá trị của customer_id chỉ tương ứng 1 giá trị customer_zip_code_prefix - quan hệ 1-1
1 giá trị của customer_id chỉ tương ứng 1 giá trị customer_city - quan hệ 1-1
1 giá trị của customer_id chỉ tương ứng 1 giá trị customer_state - quan hệ 1-1


Có thể suy đoán: 
   - 1 customer account tương ứng với 1 customer_unique_id, 
   - 1 customer_unique_id  có thể có nhiều địa chỉ nhận hàng tương, mỗi địa chỉ nhận hàng tương ứng với 1 customer_id 
    

 visualizing distribution số lượng customer_id theo location

In [7]:
customer_state = plot_bar_distribution_1d(customers,'customer_state','customer_id','Số lượng customer_id theo state','','State','#00587A')    
show(customer_state)


- hơn 90% customer_id (địa chỉ nhận hàng) tập trung tại 10 state là :SP,RJ,MG,RS,PR,SC,BA,DF,ES,GO
- SP là state tập trung nhiều customer_id nhất với 42%


### Tập dữ liệu geolocation

In [8]:
describe_data(geolocation,'geolocation')

Name of data set: geolocation
Dataset Shape: (1000163, 5)


,Name,dtypes,Missing,Uniques,Count_value,mean,std,25%,50%,75%,max,value1,value2
1,geolocation_zip_code_prefix,int64,0,19015,1000163,36574.166466,30549.335710,11075.000000,26530.000000,63504.000000,99990.000000,1037,1046
2,geolocation_lat,float64,0,717358,1000163,-21.176153,5.715866,-23.603546,-22.919377,-19.979620,45.065933,-23.5456,-23.5461
3,geolocation_lng,float64,0,717613,1000163,-46.390541,4.269748,-48.573172,-46.637879,-43.767709,121.105394,-46.6393,-46.6448
4,geolocation_city,object,0,8011,1000163,NaN,NaN,NaN,NaN,NaN,NaN,sao paulo,sao paulo
5,geolocation_state,object,0,27,1000163,NaN,NaN,NaN,NaN,NaN,NaN,SP,SP


Tập dữ liệu customers gồm 1000163 dòng với 5 cột có đặc điểm:
- Các cột đều không có dữ liệu bị thiếu

Dựa vào giá trị của các cột có thể suy luận ý nghĩa các cột như sau:
- geolocation_zip_code_prefix: zip code của 1 khu vực (mã khu vực)
- geolocation_city: tên thành phố ứng với mã khu vực
- geolocation_state: tên bang
- geolocation_lat: vĩ độ
- geolocation_lng: kinh độ
    
** Có thể suy luận đây là parent table về geographic location, dùng để xác định long, lat của customer thông qua foreign key 
customer_zip_code_prefix,customer_city,customer_state của bảng customers hoặc sellers

** Việc xác định location của customer và sellers của thể helpful trong việc optimize vấn đề chi phí về vận chuyển, kho bãi (optimize spending)

### Tập dữ liệu category

In [9]:
describe_data(category,'category')

Name of data set: category
Dataset Shape: (71, 2)


,Name,dtypes,Missing,Uniques,Count_value,value1,value2
1,product_category_name,object,0,71,71,beleza_saude,informatica_acessorios
2,product_category_name_english,object,0,71,71,health_beauty,computers_accessories


- Tập dữ liệu orders gồm 71 dòng với 2 cột(2 biến) có đặc điểm: Các biến đều chưa unqiue values và không chưa null values
Dựa vào giá trị của các cột và kiểu dữ liệu có thể suy luận đây là tập dữ liệu chưa tên danh mục sản phẩm
- Tập dữ liệu này có thể dùng để profiling customer

### Tập dữ liệu products

In [10]:
describe_data(products,'products')

Name of data set: products
Dataset Shape: (32951, 9)


,Name,dtypes,Missing,Uniques,Count_value,mean,std,25%,50%,75%,max,value1,value2
1,product_id,object,0,32951,32951,NaN,NaN,NaN,NaN,NaN,NaN,1e9e8ef04dbcff4541ed26657ea517e5,3aa071139cb16b67ca9e5dea641aaa2f
2,product_category_name,object,610,73,32341,NaN,NaN,NaN,NaN,NaN,NaN,perfumaria,artes
3,product_name_lenght,float64,610,66,32341,48.476949,10.245741,42.0,51.0,57.0,76.0,40,44
4,product_description_lenght,float64,610,2960,32341,771.495285,635.115225,339.0,595.0,972.0,3992.0,287,276
5,product_photos_qty,float64,610,19,32341,2.188986,1.736766,1.0,1.0,3.0,20.0,1,1
6,product_weight_g,float64,2,2204,32949,2276.472488,4282.038731,300.0,700.0,1900.0,40425.0,225,1000
7,product_length_cm,float64,2,99,32949,30.815078,16.914458,18.0,25.0,38.0,105.0,16,30
8,product_height_cm,float64,2,102,32949,16.937661,13.637554,8.0,13.0,21.0,105.0,10,18
9,product_width_cm,float64,2,95,32949,23.196728,12.079047,15.0,20.0,30.0,118.0,14,20


Tập dữ liệu orders gồm 32951 dòng với 9 cột(9 biến) có đặc điểm:

- 7/9 đều thuộc kiểu float
- ngoại trừ product_id, các biến khác đều có thể chưa giá trị null
Dựa vào giá trị của các cột và kiểu dữ liệu có thể suy luận ý nghĩa các cột như sau:

- product_id: mã sản phẩm
- product_category_name: tên danh mục
- product_name_lenght: độ dài tên sp
- product_description_lenght: độ dài mô tả sp
- product_photos_qty: số lượng ảnh sp
- product_weight_g: khối lượng (g)
- product_length_cm: dài(cm)
- product_height_cm: cao (cm)
- product_width_cm: rông(cm)


Tập dữ liệu này có thể dùng để ước lượng chi phí vận chuyển theo kích thước sản phẩm, nhóm các đơn hàng có kích thước vận chuyển tương đồng để điều phối phương tiện vận chuyển cho phù hợp, đồng thời giảm thiểu quãng đường giao vận


### Tập dữ liệu order_items

In [11]:
describe_data(order_items,'order_items') 

Name of data set: order_items
Dataset Shape: (112650, 7)


,Name,dtypes,Missing,Uniques,Count_value,mean,std,25%,50%,75%,max,value1,value2
1,order_id,object,0,98666,112650,NaN,NaN,NaN,NaN,NaN,NaN,00010242fe8c5a6d1ba2dd792cb16214,00018f77f2f0320c557190d7a144bdd3
2,order_item_id,int64,0,21,112650,1.197834,0.705124,1.00,1.00,1.00,21.00,1,1
3,product_id,object,0,32951,112650,NaN,NaN,NaN,NaN,NaN,NaN,4244733e06e7ecb4970a6e2683c13e61,e5f2d52b802189ee658865ca93d83a8f
4,seller_id,object,0,3095,112650,NaN,NaN,NaN,NaN,NaN,NaN,48436dade18ac8b2bce089ec2a041202,dd7ddc04e1b6c2c614352b383efe2d36
5,shipping_limit_date,object,0,93318,112650,NaN,NaN,NaN,NaN,NaN,NaN,2017-09-19 09:45:35,2017-05-03 11:05:13
6,price,float64,0,5968,112650,120.653739,183.633928,39.90,74.99,134.90,6735.00,58.9,239.9
7,freight_value,float64,0,6999,112650,19.990320,15.806405,13.08,16.26,21.15,409.68,13.29,19.93


Tập dữ liệu customers gồm 112650 dòng với 7 cột (7 biến) có đặc điểm:
- Các cột đều không có dữ liệu bị thiếu

Dựa vào giá trị của các cột có thể suy luận ý nghĩa các cột như sau:
- order_id: mã đơn hàng, kiểu object(string)
- order_item_id: số thứ tự sản phẩm trong cùng 1 order, int
- product_id: mã sản phẩm, object(string), foreign key- references tới bảng products(product_id) để biết thông tin chi tiết sản phẩm
- seller_id: id người bán, object(string),foreign key- references tới bảng seller(seller_id) để biết thông tin chi tiết location người bán
- shipping_limit_date: ngày dự kiến giao , datetime
- price: giá sản phẩm, float
- freight_value: phí vận chuyển, float

In [12]:
for var2 in ('product_id','seller_id','price','freight_value'):
    check_references_relationship(order_items,'order_id',var2)

1 giá trị của order_id có thể có nhiều giá trị product_id - quan hệ 1-nhiều
1 giá trị của order_id có thể có nhiều giá trị seller_id - quan hệ 1-nhiều
1 giá trị của order_id có thể có nhiều giá trị price - quan hệ 1-nhiều
1 giá trị của order_id có thể có nhiều giá trị freight_value - quan hệ 1-nhiều


In [13]:
for var2 in ('seller_id','price','freight_value'):
    check_references_relationship(order_items,'product_id',var2)

1 giá trị của product_id có thể có nhiều giá trị seller_id - quan hệ 1-nhiều
1 giá trị của product_id có thể có nhiều giá trị price - quan hệ 1-nhiều
1 giá trị của product_id có thể có nhiều giá trị freight_value - quan hệ 1-nhiều


- Như vậy cùng 1 mã sản phẩm có thể được bán bởi nhiều sellers khác nhau
- 1 mã sản phẩm có thể có nhiều mức giá khác nhau

In [14]:
order_line = order_items.groupby(['order_id','product_id','price','freight_value']).agg({'order_item_id':'count'}).reset_index()
order_line.rename(columns={'order_item_id':'item_quantity'},inplace=True)
order_line['product_subtotal'] = order_line['item_quantity']*order_line['price']
order_line.head(3)

,order_id,product_id,price,freight_value,item_quantity,product_subtotal
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.9,13.29,1,58.9
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.9,19.93,1,239.9
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,199.0,17.87,1,199.0


In [15]:
# Tính basket size và basket value
order_sum = order_line.groupby(['order_id']).agg({'item_quantity':sum,'product_subtotal':sum,'freight_value':sum}).reset_index()
order_sum.rename(columns={'product_subtotal':'subtotal'},inplace=True)
describe_data(order_sum,'Subtotal của đơn hàng')

Name of data set: Subtotal của đơn hàng
Dataset Shape: (98666, 4)


,Name,dtypes,Missing,Uniques,Count_value,mean,std,25%,50%,75%,max,value1,value2
1,order_id,object,0,98666,98666,NaN,NaN,NaN,NaN,NaN,NaN,00010242fe8c5a6d1ba2dd792cb16214,00018f77f2f0320c557190d7a144bdd3
2,item_quantity,int64,0,17,98666,1.141731,0.538452,1.00,1.00,1.00,21.00,1,1
3,subtotal,float64,0,7760,98666,137.754076,210.645145,45.90,86.90,149.90,13440.00,58.9,239.9
4,freight_value,float64,0,7126,98666,20.877352,16.967415,13.47,16.66,22.14,1002.29,13.29,19.93



- TB 1 đơn hàng có khoảng 1,14 item ; 75% đơn hàng chỉ chứa 1 item; 21 là số lượng item lớn nhất trong 1 đơn hàng từng có
- TB 1 đơn hàng có giá trị (sản phẩm) khoảng 137,75 (đơn vị tiền-đvt), 50% số đơn hàng có giá trị dưới 86,9; 75% số đơn hàng có giá trị dưới 149,9 trong khi std = 210 cho thấy độ phân tán của giá trị rộng.
- TB 1 đơn hàng cost 20,8 dvt chi phí vận chuyển, 50% số đơn có mức chi phí vận chuyển dưới 16,66,  std = 16.96 cho thấy mức độ phân tán của giá trị tương đối tập trung

In [16]:
# Do subtotal std lơn, độ phân tán rộng, ta lấy các đơn hàng có subtotal nằm ngoài khoảng phân phối 99% để tìm giá trị outlier
quantile = 0.99
q_high = order_sum['subtotal'].quantile(quantile)
print('q_high:',q_high)
# q_low = order_sum['subtotal'].quantile(0.01)
# print('q_high:',q_high,'\n','q_low:',q_low)

q_high: 998.8999999999999


In [17]:
# Vẽ plot thể hiện distribution của subtotal, freight_value, nằm trong, và ngoài quantile(q_high,q_low)
def plot_distribution(bins,val,title,x_label,density=False):
    hist, edges = np.histogram(val, density=density, bins=bins)
    p = figure(plot_width = 500,plot_height=300,title=title, toolbar_location=None,tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="#00587A", line_color="white", alpha=0.5)
    p.axis.axis_line_color = 'silver'
    p.axis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label = 'Count'
    p.xaxis.axis_label = '{} [bins={}]'.format(x_label,bins)
    p.xgrid.grid_line_color=None
    p.xaxis.major_tick_in=0
    if density==False:
        p.yaxis.formatter=NumeralTickFormatter(format="0,0")
        p.yaxis.axis_label = 'Count'
    else:
        p.yaxis.formatter=NumeralTickFormatter(format="0.0000")
        p.yaxis.axis_label = 'Desity'
    p.xaxis.formatter=NumeralTickFormatter(format="0,0")
    return(p)
order_subtotal_dist=plot_distribution(100,order_sum.loc[order_sum['subtotal']<q_high]['subtotal'].tolist(),
                                      'Phân phối Giá trị đơn hàng quantite =<{}'.format(quantile),'Giá trị đơn hàng')
order_subtotal_dist_ouliner=plot_distribution(50,order_sum.loc[order_sum['subtotal']>=q_high]['subtotal'].tolist(),
                                              'Phân phối Giá trị đơn hàng quantite >{}'.format(quantile),'Giá trị đơn hàng')
order_item_quantity_dist = plot_distribution(100,order_sum.loc[order_sum['subtotal']<q_high]['freight_value'].tolist(),
                                             'Phân phối Phí vận chuyển với quantite subtotal =<{}'.format(quantile),'Phí vận chuyển')
order_item_quantity_dist_ouliner = plot_distribution(50,order_sum.loc[order_sum['subtotal']>=q_high]['freight_value'].tolist(),
                                                     'Phân phối Phí vận chuyển với quantite subtotal >{}'.format(quantile),'Phí vận chuyển')

show(row(column(order_subtotal_dist,order_item_quantity_dist),column(order_subtotal_dist_ouliner,order_item_quantity_dist_ouliner)))


- 99% giá trị subtotal của đơn hàng có giá trị < 998.89
- 0.01% giá trị subtotal > 998.89, phân bố tập trung trong khoảng giá trị từ 1000-3000
- Phí vận chuyển của các đơn hàng có subtotal nằm trong khoảng quantite 99% có giá trị phân bố chủ yêu trong khoảng từ 0-dưới 100
- Phí vận chuyển của các đơn hàng có subtotal là outliers có giá trị phân bố chủ yếu từ 0-200, nằm tập trung nhất cũng trong khoảng từ  0-100
- Có thể thấy Phí vận chuyển không có sự tương quan rõ ràng với giá trị đơn, những đơn hàng có subtotal cực lớn không dẫn đến mức phí vận chuyển quá lớn kèm theo vì distribution vẫn nằm tập trung chủ yếu trong khoảng 0-100

In [18]:
# Kiểm tra tỷ trọng gross sales theo danh mục hàng hóa
order_items_category = order_items.merge(products, how = 'left', left_on = 'product_id', right_on= 'product_id')
order_items_category = order_items_category.merge(category, how = 'left', left_on = 'product_category_name',right_on = 'product_category_name')
order_items_category = order_items_category.groupby(['product_category_name_english']).agg({'order_item_id':'count','price':'sum'}).reset_index()
order_items_category.rename(columns = {'order_item_id':'item quantity', 'price':'item gross sale value'},inplace=True)
order_items_category['% quantity']= order_items_category['item quantity']/order_items_category['item quantity'].sum()
order_items_category['% sales value']= order_items_category['item gross sale value']/order_items_category['item gross sale value'].sum()
order_items_category = order_items_category.sort_values(by=['item quantity','item gross sale value'],ascending = False).reset_index(drop=True)
order_items_category


,product_category_name_english,item quantity,item gross sale value,% quantity,% sales value
0,bed_bath_table,11115,1036988.68,0.100114,0.077349
1,health_beauty,9670,1258681.34,0.087099,0.093885
2,sports_leisure,8641,988048.97,0.077831,0.073699
3,furniture_decor,8334,729762.49,0.075066,0.054433
4,computers_accessories,7827,911954.32,0.070499,0.068023
...,...,...,...,...,...
66,arts_and_craftmanship,24,1814.01,0.000216,0.000135
67,la_cuisine,14,2054.99,0.000126,0.000153
68,cds_dvds_musicals,14,730.00,0.000126,0.000054
69,fashion_childrens_clothes,8,569.85,0.000072,0.000043


In [19]:
order_items_category_top = order_items_category.loc[order_items_category['% quantity']>0.01]
count_items_category_plot = plot_bar_distribution_1d(order_items_category_top,'product_category_name_english','item quantity','Số lượng sản phẩm ','Số lượng sản phẩm','','#00587A',count=False)
show(count_items_category_plot)


- Phần lớn các sản phẩm được đặt mua thuộc nhóm hàng chăm sóc sứ khỏe, làm đẹp, và các vật dụng thông thường cho chăm sóc nhà cửa, vì vậy mà ABV không lớn

In [20]:
# Tính tổng giá trị sales theo sellers
order_line_seller = order_items.groupby(['order_id','seller_id','product_id','price','freight_value']).agg({'order_item_id':'count'}).reset_index()
order_line_seller.rename(columns={'order_item_id':'item_quantity'},inplace=True)
order_line_seller['product_subtotal'] = order_line_seller['item_quantity']*order_line_seller['price']

order_sum_seller = order_line_seller.groupby(['seller_id']).agg({'item_quantity':sum,'product_subtotal':sum,'freight_value':sum}).reset_index()
order_sum_seller.rename(columns={'product_subtotal':'sales revenue'},inplace=True)
describe_data(order_sum_seller,'Doanh số theo sellers')

Name of data set: Doanh số theo sellers
Dataset Shape: (3095, 4)


,Name,dtypes,Missing,Uniques,Count_value,mean,std,25%,50%,75%,max,value1,value2
1,seller_id,object,0,3095,3095,NaN,NaN,NaN,NaN,NaN,NaN,0015a82c2db000af6aaaf3ae2ecb0532,001cca7ae9ae17fb1caed9dfb1094831
2,item_quantity,int64,0,257,3095,36.397415,119.193461,2.000,8.00,24.000,2033.00,3,239
3,sales revenue,float64,0,2786,3095,4391.484233,13921.997192,208.850,821.48,3280.830,229472.63,2685,25080
4,freight_value,float64,0,2925,3095,665.552433,2076.656109,41.935,138.14,500.075,38677.08,63.06,7553.01


- 75% seller có doanh số <= 3280.83 trong khi std lại = 13921.997192 cho thấy giá trị sales revenue theo seller phân bố rất phân tán

In [21]:
#Tìm các giá trị outliners
quantile_high_s = 0.99
quantile_low_s = 0.01
q_high_s = order_sum_seller['sales revenue'].quantile(quantile_high_s)
q_low_s =  order_sum_seller['sales revenue'].quantile(quantile_low_s)
print('q_high:',q_high_s,'\n','q_low:',q_low_s)

q_high: 55108.71619999998 
 q_low: 18.9


In [22]:
# Phân phối doanh thu theo người bán đã loại bỏ các giá trị ouliers nằm ngoài khoảng 0.01 và 0.99
order_sum_seller_exclude_outlier = order_sum_seller.loc[(order_sum_seller['sales revenue']<=q_high_s)
                                                        & (order_sum_seller['sales revenue']>=q_low_s)]

seller_sales_count = plot_distribution(30,order_sum_seller_exclude_outlier['sales revenue'],'Phân phối Sales Revenue theo Seller','',density=False)
seller_sales_dens = plot_distribution(30,order_sum_seller_exclude_outlier['sales revenue'],'Phân phối Sales Revenue theo Seller','',density=True)
seller_sales_plot = row(seller_sales_count,seller_sales_dens)

show(seller_sales_plot)

- Phần lớn sellers có tổng doanh thu tập trung trong khoảng giá trị <10000 trong đó phân bố tập trung nhất (density) trong khoảng <6000

In [23]:
# Tính TB basket value (tb giá trị đơn hàng) theo sellers
order_line_seller = order_items.groupby(['order_id','seller_id','product_id','price','freight_value']).agg({'order_item_id':'count'}).reset_index()
order_line_seller.rename(columns={'order_item_id':'item_quantity'},inplace=True)
order_line_seller['product_subtotal'] = order_line_seller['item_quantity']*order_line_seller['price']

abv_seller = order_line_seller.groupby(['seller_id']).agg({'product_subtotal':sum,'order_id':pd.Series.nunique}).reset_index()
abv_seller['abv'] = abv_seller['product_subtotal']/abv_seller['order_id']
quantile_high_abv = 0.99
quantile_low_abv = 0.01
q_high_s = abv_seller['abv'].quantile(quantile_high_abv)
q_low_s =  abv_seller['abv'].quantile(quantile_low_abv)
print('q_high:',q_high_s,'\n','q_low:',q_low_s)
abv_seller_exclude_outlier = abv_seller.loc[(abv_seller['abv']<=q_high_s)
                                                        & (abv_seller['abv']>=q_low_s)]

abv_seller_count = plot_distribution(30,abv_seller_exclude_outlier['abv'],'Phân phối Average basket value theo Seller','',density=False)
abv_seller_dens = plot_distribution(30,abv_seller_exclude_outlier['abv'],'Phân phối Average basket value theo Seller','',density=True)
abv_seller_plot = row(abv_seller_count,abv_seller_dens)

show(abv_seller_plot)


q_high: 1599.0574 
 q_low: 13.9994


- ABV của sellers nằm phân bố tập trung trong khoảng từ 13.9 - 500

### Tập dữ liệu payments

In [24]:
describe_data(payments,'payments')

Name of data set: payments
Dataset Shape: (103886, 5)


,Name,dtypes,Missing,Uniques,Count_value,mean,std,25%,50%,75%,max,value1,value2
1,order_id,object,0,99440,103886,NaN,NaN,NaN,NaN,NaN,NaN,b81ef226f3fe1789b1e8b2acac839d17,a9810da82917af2d9aefd1278f1dcfa0
2,payment_sequential,int64,0,29,103886,1.092679,0.706584,1.00,1.0,1.0000,29.00,1,1
3,payment_type,object,0,5,103886,NaN,NaN,NaN,NaN,NaN,NaN,credit_card,credit_card
4,payment_installments,int64,0,24,103886,2.853349,2.687051,1.00,1.0,4.0000,24.00,8,1
5,payment_value,float64,0,29077,103886,154.100380,217.494064,56.79,100.0,171.8375,13664.08,99.33,24.39


In [25]:
# Kiểm tra 1 vài records để hiểu hơn về cấu trúc bảng
count_payment_sequential = payments.groupby(['order_id']).agg({'payment_sequential':'count'})
ls_orders = count_payment_sequential.loc[count_payment_sequential['payment_sequential']>2][:2].index.tolist()
payments.loc[payments['order_id'].isin(ls_orders)].sort_values(by=['order_id','payment_sequential'])


,order_id,payment_sequential,payment_type,payment_installments,payment_value
16053,009ac365164f8e06f59d18a08045f6c4,1,credit_card,1,0.88
16459,009ac365164f8e06f59d18a08045f6c4,2,voucher,1,4.50
73837,009ac365164f8e06f59d18a08045f6c4,3,voucher,1,8.25
32058,009ac365164f8e06f59d18a08045f6c4,4,voucher,1,5.45
285,009ac365164f8e06f59d18a08045f6c4,5,voucher,1,8.75
15298,009ac365164f8e06f59d18a08045f6c4,6,voucher,1,4.17
34559,00bd50cdd31bd22e9081e6e2d5b3577b,1,credit_card,1,4.88
20650,00bd50cdd31bd22e9081e6e2d5b3577b,2,voucher,1,40.46
8869,00bd50cdd31bd22e9081e6e2d5b3577b,3,voucher,1,40.46


Tập dữ liệu customers gồm 103886 dòng với 5 cột (5 biến) có đặc điểm:
- Các cột đều không có dữ liệu bị thiếu

Dựa vào giá trị của các cột và kiểu dữ liệu có thể suy luận ý nghĩa các cột như sau:
- order_id: mã đơn hàng
- payment_sequential: số stt phương thức thanh toán được sử dụng để thanh toán 1 order tương ứng
- payment_type: tên phương thức thanh toán
- payment_installments: số lượng kỳ thanh toán trả góp
- payment_value: giá trị thanh toán
1 order có thể sử dụng nhiều phương thức thanh toán, tổng giá trị payment value theo mỗi payment_sequential bằng tổng giá trị thanh toán của đơn hàng

In [26]:
payment_method_sum = payments.groupby(['payment_type']).agg({'order_id':pd.Series.nunique,'payment_value':sum})
payment_method_sum


,order_id,payment_value
payment_type,,
boleto,19784,2.869361e+06
credit_card,76505,1.254208e+07
debit_card,1528,2.179898e+05
not_defined,3,0.000000e+00
voucher,3866,3.794369e+05


In [27]:
payment_method_order = plot_bar_distribution_1d(payments,'payment_type','order_id','Số lượng đơn theo phương thức thanh toán','Số đơn','','#00587A')
payment_method_value = plot_bar_distribution_1d(payments,'payment_type','payment_value','Tổng giá trị thanh toán theo phương thức thanh toán','Giá trị thanh toán','','#FFBD39')
show(column(payment_method_order,payment_method_value))


- 75.2% số đơn thanh toán bằng credit card (tương ứng 62.2% tổng giá trị thanh toán)
- 19.5% số đơn thanh toán qua boleto (chiếm  25.7% giá trị thanh toán)
- 3.8% số đơn có sử dụng voucher với giá trị chiểm 8.6% tổng giá trị thanh toán

Có thể suy luận những đơn hàng có giá trị lớn có xu hướng sử dụng voucher và thanh toán qua boleto

### Tập dữ liệu reviews

In [28]:
describe_data(reviews,'reviews')

Name of data set: reviews
Dataset Shape: (100000, 7)


,Name,dtypes,Missing,Uniques,Count_value,mean,std,25%,50%,75%,max,value1,value2
1,review_id,object,0,99173,100000,NaN,NaN,NaN,NaN,NaN,NaN,7bc2406110b926393aa56f80a40eba40,80e641a11e56f04c1ad469d5645fdfde
2,order_id,object,0,99441,100000,NaN,NaN,NaN,NaN,NaN,NaN,73fc7af87114b39712e6da79b0a377eb,a548910a1c6147796b98fdf73dbeba33
3,review_score,int64,0,5,100000,4.07089,1.359663,4.0,5.0,5.0,5.0,4,5
4,review_comment_title,object,88285,4600,11715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,review_comment_message,object,58247,36921,41753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,review_creation_date,object,0,637,100000,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-18 00:00:00,2018-03-10 00:00:00
7,review_answer_timestamp,object,0,99010,100000,NaN,NaN,NaN,NaN,NaN,NaN,2018-01-18 21:46:59,2018-03-11 03:05:13


In [29]:
# Kiểm tra 1 vài records để hiểu hơn về cấu trúc bảng
# 1 order_id  cho nhiều  review_id
count_reviews = reviews.groupby(['order_id']).agg({'review_id':'count'})
ls_orders = count_reviews.loc[count_reviews['review_id']>2][:1].index.tolist()
reviews.loc[reviews['order_id'].isin(ls_orders)].sort_values(by=['order_id','review_id'])


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
70005,405eb2ea45e1dbe2662541ae5b47e2aa,03c939fd7fd3b38f8485a0f95798f1f6,3,NaN,Seria ótimo se tivesem entregue os 3 (três) pe...,2018-03-06 00:00:00,2018-03-06 19:50:32
8329,b04ed893318da5b863e878cd3d0511df,03c939fd7fd3b38f8485a0f95798f1f6,3,NaN,Um ponto negativo que achei foi a cobrança de ...,2018-03-20 00:00:00,2018-03-21 02:28:23
51931,f4bb9d6dd4fb6dcc2298f0e7b17b8e1e,03c939fd7fd3b38f8485a0f95798f1f6,4,NaN,NaN,2018-03-29 00:00:00,2018-03-30 00:29:09


In [30]:
# 1 review_id cho nhiều order_id
count_reviews = reviews.groupby(['review_id']).agg({'order_id':'count'})
ls_orders = count_reviews.loc[count_reviews['order_id']>2][:1].index.tolist()
reviews.loc[reviews['review_id'].isin(ls_orders)].sort_values(by=['review_id','order_id'])


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
40245,08528f70f579f0c830189efc523d2182,03310aa823a66056268a3bab36e827fb,1,Produto errado,Entrega do produto diferente do solicitado\r\n...,2018-08-03 00:00:00,2018-08-06 00:09:52
75424,08528f70f579f0c830189efc523d2182,53c71d3953507c6239ff73917ed358c9,1,Produto errado,Entrega do produto diferente do solicitado\r\n...,2018-08-03 00:00:00,2018-08-06 00:09:52
8462,08528f70f579f0c830189efc523d2182,7813842ae95e8c497fc0233232ae815a,1,Produto errado,Entrega do produto diferente do solicitado\r\n...,2018-08-03 00:00:00,2018-08-06 00:09:52


Tập dữ liệu customers gồm 100000 dòng với 7 cột (7 biến) có đặc điểm:
- review_comment_title và review_comment_message chứa null values

Dựa vào giá trị của các cột và kiểu dữ liệu có thể suy luận ý nghĩa các cột như sau:
- review_id: id của review
- order_id: mã đơn hàng, 1 order_id có thể có nhiều review_id
- review_score: điểm đánh giá review
- review_comment_title: tiêu đề chung của review comment
- review_comment_message: nội dung của review comment
- review_creation_date: ngày tạo review
- review_answer_timestamp: ngày giờ trả lời review

In [31]:
avg_score = reviews.groupby(['order_id']).agg({'review_score':'mean'}).reset_index()
print('Điểm TB 1 đơn hàng :',avg_score['review_score'].mean())
reviews['review_score_str'] = reviews['review_score'].astype('str')
reviews_score = plot_bar_distribution_1d(reviews,'review_score_str','review_id','Thống kê điểm review','Số lượng review đánh giá','Thang điểm','#00587A')

show(reviews_score)

Điểm TB 1 đơn hàng : 4.071234869587662


- 76.7% số review đánh giá  từ 4 điểm trở lên cho chất lượng dịch vụ và sản phẩm, tuy nhiên số review cho đánh giá 1 điểm chiếm tỷ trọng tương đối lớn (11.8%) cho thấy chất lượng dịch vụ cần thêm nhiều cải thiện 

### Tập dữ liệu orders

In [32]:
describe_data(orders,'orders')

Name of data set: orders
Dataset Shape: (99441, 8)


,Name,dtypes,Missing,Uniques,Count_value,value1,value2
1,order_id,object,0,99441,99441,e481f51cbdc54678b7cc49136f2d6af7,53cdb2fc8bc7dce0b6741e2150273451
2,customer_id,object,0,99441,99441,9ef432eb6251297304e76186b10a928d,b0830fb4747a6c6d20dea0b8c802d7ef
3,order_status,object,0,8,99441,delivered,delivered
4,order_purchase_timestamp,object,0,98875,99441,2017-10-02 10:56:33,2018-07-24 20:41:37
5,order_approved_at,object,160,90733,99281,2017-10-02 11:07:15,2018-07-26 03:24:27
6,order_delivered_carrier_date,object,1783,81018,97658,2017-10-04 19:55:00,2018-07-26 14:31:00
7,order_delivered_customer_date,object,2965,95664,96476,2017-10-10 21:25:13,2018-08-07 15:27:45
8,order_estimated_delivery_date,object,0,459,99441,2017-10-18 00:00:00,2018-08-13 00:00:00


Tập dữ liệu orders gồm 99441 dòng với 8 cột (8 biến) có đặc điểm:
- đều thuộc kiểu object (string hoặc datetime)
- order_approved_at,order_delivered_carrier_date,order_delivered_customer_date có chứa null values

Dựa vào giá trị của các cột và kiểu dữ liệu có thể suy luận ý nghĩa các cột như sau:
- order_id: 
- customer_id: mã khách hàng 
- order_status: trạng thái đơn hàng
- order_purchase_timestamp: thời gian đặt hàng
- order_approved_at: thời gian xác nhận đơn hàng
- order_delivered_carrier_date: thời gian đơn hàng bắt đầu được vận chuyển
- order_delivered_customer_date: thời gian khách hàng nhận hàng
- order_estimated_delivery_date: thời gian dự kiến tối đa

In [33]:
# Kiểm tra các trạng thái có thể có của đơn hàng
orders.groupby(['order_status']).agg({'order_id':'count'}).rename(columns={'order_id':'number of order'})

,number of order
order_status,
approved,2
canceled,625
created,5
delivered,96478
invoiced,314
processing,301
shipped,1107
unavailable,609


Tính thời gian process của 1 đơn hàng theo từng stage của các order đã được delivered hoặc shipped
- time stage 1 : Thời gian từ order_purchase_timestamp đến order_approved_at
- time stage 2 : Thời gian từ order_approved_at đến order_delivered_carrier_date
- time stage 3 : Thời gian từ order_delivered_carrier_date đến order_delivered_customer_date

In [34]:
#Tính thời gian process của 1 đơn hàng theo từng stage của các order đã được delivered hoặc shipped, loại bỏ các đơn hàng bị thiếu 1 trong các mốc thời gian 
orders_delivered = orders.loc[orders['order_status'].isin(['delivered','shipped'])]
orders_delivered = orders_delivered.loc[((orders_delivered['order_approved_at'].isnull()==False)&(orders_delivered['order_delivered_carrier_date'].isnull()==False)&(orders_delivered['order_delivered_customer_date'].isnull()==False))]

orders_delivered['time_stage1']= ((pd.to_datetime(orders_delivered['order_approved_at']) - pd.to_datetime(orders_delivered['order_purchase_timestamp']))\
                        /pd.Timedelta(minutes=1)).round()
orders_delivered['time_stage2']= ((pd.to_datetime(orders_delivered['order_delivered_carrier_date']) - pd.to_datetime(orders_delivered['order_approved_at']))\
                        /pd.Timedelta(minutes=1)).round()
orders_delivered['time_stage3']= ((pd.to_datetime(orders_delivered['order_delivered_customer_date']) - pd.to_datetime(orders_delivered['order_delivered_carrier_date']))\
                        /pd.Timedelta(minutes=1)).round()
orders_delivered['time_total']= ((pd.to_datetime(orders_delivered['order_delivered_customer_date']) - pd.to_datetime(orders_delivered['order_approved_at']))\
                        /pd.Timedelta(minutes=1)).round()

In [35]:
print('TB Thời gian stage 1 (phút):',orders_delivered['time_stage1'].mean(),'\n',
'TB Thời gian stage 2 (phút):',orders_delivered['time_stage2'].mean(),'\n',
'TB Thời gian stage 3 (phút):',orders_delivered['time_stage3'].mean(),'\n',
'TB Tổng Thời gian xử lý (phút):',orders_delivered['time_total'].mean())


TB Thời gian stage 1 (phút): 616.6620703955213 
 TB Thời gian stage 2 (phút): 4030.6502825151624 
 TB Thời gian stage 3 (phút): 13435.812389197035 
 TB Tổng Thời gian xử lý (phút): 17466.408439168525


- Thời gian xử lý của stage 1 còn tương tối lớn (khoảng 10 tiếng từ lúc kh đặt đơn cho đến khi đơn được xác nhận)
- Thời gian chuẩn bị hàng : từ lúc đơn được xác nhận đến khi đơn vị vận chuyển nhận hàng để tiến hành giao vận khoảng 2,8 ngày
- Thời gian giao vận cho 1 đơn hàng tương đối lớn : > 9 ngày
- Tổng thời gian TB cho 1 đơn hàng được xử lý là :>12 ngày

Thời gian xử lý đơn hàng lớn có thể xuất phát từ khoảng cách địa lý, break down thời gian xử lý đơn theo vùng địa lý để có cái nhìn rõ hơn

In [36]:
orders_delivered = orders_delivered.merge(customers, how = 'left', left_on = 'customer_id',right_on = 'customer_id')


In [37]:
time_stage3_by_state = orders_delivered.groupby(['customer_state']).agg({'time_stage3':'mean'
                                                                                  }).rename(columns={'time_stage3':'Time'})
time_stage3_by_state['type'] = 'Thời gian vận chuyển'
time_total_by_state = orders_delivered.groupby(['customer_state']).agg({'time_total':'mean'
                                                                                  }).rename(columns={'time_total':'Time'})
time_total_by_state['type'] = 'Tổng thời gian xử lý đơn'
time_by_state = time_stage3_by_state.append(time_total_by_state).reset_index()
time_by_state['color'] = time_by_state['type'].apply(lambda x: '#00587A' if x == 'Thời gian vận chuyển' else '#FFBD39' )
time_by_state_plot = plot_bar_distribution_2d(time_by_state,'customer_state','type','Time','Thời gian vận chuyển và Tổng thời gian xử lý đơn','Phút','',sort=False,pct=False)
show(time_by_state_plot)


- Phần lớn các state đều có TB tổng thời gian xử lý đơn > 20000 phút ( gần 14 ngày)
- Các state có tổng thời gian xử lý đơn lớn nhất đều có thời gian giao vận xếp nhiều nhất (RR,AP,AM,AL,PA,SE,MA..)

### Tập dữ liệu sellers

In [38]:
describe_data(sellers,'sellers')

Name of data set: sellers
Dataset Shape: (3095, 4)


,Name,dtypes,Missing,Uniques,Count_value,mean,std,25%,50%,75%,max,value1,value2
1,seller_id,object,0,3095,3095,NaN,NaN,NaN,NaN,NaN,NaN,3442f8959a84dea7ee197c632cb2df15,d1b65fc7debc3361ea86b5f14c68d2e2
2,seller_zip_code_prefix,int64,0,2246,3095,32291.059451,32713.45383,7093.5,14940.0,64552.5,99730.0,13023,13844
3,seller_city,object,0,611,3095,NaN,NaN,NaN,NaN,NaN,NaN,campinas,mogi guacu
4,seller_state,object,0,23,3095,NaN,NaN,NaN,NaN,NaN,NaN,SP,SP


Tập dữ liệu orders gồm 3095 dòng với 4 cột(4 biến) có đặc điểm: Các biến đều không chứa null values
Dựa vào giá trị của các cột và kiểu dữ liệu có thể suy luận đây là tập dữ liệu chứa thông tin về location của seller

In [39]:
seller_state = plot_bar_distribution_1d(sellers,'seller_state','seller_id','Số lượng seller_id theo state','','State','#00587A')    
show(seller_state)


 - Số lượng seller tập trung chủ yếu tại bang SP và PR

So sánh tỷ trọng phân bố seller, customer theo state, loại những state có tỷ trọng seller hoặc customer < 1% để dễ quan sát hơn

In [40]:
state_seller = sellers.groupby(['seller_state']).agg({'seller_id':pd.Series.nunique}).reset_index()
state_seller['pct']= state_seller['seller_id']/state_seller['seller_id'].sum()
state_seller= state_seller[['seller_state','pct']]
state_customer = customers.groupby(['customer_state']).agg({'customer_id':pd.Series.nunique}).reset_index()
state_customer['pct']= state_customer['customer_id']/state_customer['customer_id'].sum()
state_customer= state_customer[['customer_state','pct']]
geolocation_2 = geolocation.groupby(['geolocation_state',]).count().reset_index()[['geolocation_state']]
geolocation_s = geolocation_2.merge(state_seller, how='left',left_on='geolocation_state', right_on = 'seller_state' ).fillna(0)[['geolocation_state','pct']]
geolocation_s['type'] = 'seller'
geolocation_c = geolocation_2.merge(state_customer, how='left',left_on='geolocation_state', right_on = 'customer_state' ).fillna(0)[['geolocation_state','pct']]
geolocation_c['type']= 'customer'
dist_sel_cus = geolocation_s.append(geolocation_c,ignore_index = True)
dist_sel_cus= dist_sel_cus.loc[dist_sel_cus['pct']>0.01]
dist_sel_cus['color'] =  dist_sel_cus['type'].apply(lambda x: '#00587A' if x == 'seller' else '#FFBD39' )

dist_sel_cus_plot = plot_bar_distribution_2d(dist_sel_cus,'geolocation_state','type','pct','So sảnh tỷ trọng phân bố seller, customer theo state','% tỷ trọng trong tổng số seller/customer','State',sort=False)
show(dist_sel_cus_plot)


- Nhìn chung, lượng sellers và customer tập trung chủ yếu tại các bang SP, Mg, PR, RJ. Trong đó SP là bang chiếm đến 59.7% số lượng nhà cung ứng và 42% tập khách hàng
- Tuy nhiên cần chú ý tới các bang có tỷ trọng khách hàng đáng kể tuy nhiên lượng người bán vẫn còn thấp, cho thấy dấu hiệu khả năng cung chưa đủ đáp ứng cầu, tỷ lệ cạnh tranh seller còn thấp, đây là những vùng có nhiều room để mở rộng số lượng và quy mô của nhà cung ứng (seller)

So sánh thêm với tổng giá trị sales, và số lượng đơn hàng theo các state trên để tìm xem state nào đang có nhiều cơ hội để scale up

In [41]:
# MApping bảng dữ liệu orders với customer để tìm điểm đến của order theo state của customer
orders_state = orders.merge(customers, how = 'left', left_on = 'customer_id',right_on = 'customer_id')
# MApping bảng dữ liệu orders_state với order_sum đã được tạo để trên, sau đó tính tổng giá trị mua hàng của khách hàng theo state
order_state = order_sum.merge(orders_state, how = 'left', left_on = 'order_id',right_on = 'order_id')

In [42]:
#Chỉ lấy giá trị của đơn được vận chuyển: có order_status= 'delivered',shipped và chỉ lấy danh sách state ở trên ( tỷ trọng seller hoặc customer < 1% để dễ quan sát hơn) để so sanh
list_state_centered = dist_sel_cus['geolocation_state'].unique().tolist()
order_state_shipped = order_state.loc[(order_state['order_status'].isin(['delivered','shipped']))&(order_state['customer_state'].isin(list_state_centered))].\
groupby(['customer_state']).agg({'order_id':pd.Series.nunique,'subtotal':sum})
order_state_shipped['pct'] = order_state_shipped['order_id']/order_state_shipped['order_id'].sum()
order_state_sipped_o =  pd.DataFrame(order_state_shipped['pct']).reset_index()
order_state_sipped_o['type'] ='order' 

order_state_shipped['pct_s'] = order_state_shipped['subtotal']/order_state_shipped['subtotal'].sum()
order_state_sipped_s =  pd.DataFrame(order_state_shipped['pct_s']).reset_index().rename(columns={'pct_s':'pct'})
order_state_sipped_s['type'] ='subtotal' 
order_state_shipped = order_state_sipped_o.append(order_state_sipped_s)
order_state_shipped['color'] =  order_state_shipped['type'].apply(lambda x: '#00587A' if x == 'order' else '#FFBD39' )

order_state_shipped_plot = plot_bar_distribution_2d(order_state_shipped,'customer_state','type','pct','So sảnh tỷ trọng phân bố doanh số, đơn hàng theo state','% tỷ trọng trong tổng orders/subtotal','State',sort=False)
show(column(order_state_shipped_plot,dist_sel_cus_plot))

Nhìn vào 2 chart trên có thể chia các state thành 3 nhóm:
- Nhóm 1: (SP) Đã phát triển bão hòa, số lượng seller và customer vướt trội, tuy nhiên seller > customer, order>subtotal => ABV cúa state (average basket value) < Trung Bình ABV toàn bộ các order
- Nhóm 2:(MG,PR,RJ,RS,SC) Đang phát triển, đã phát triển được 1 tập seller và customer, tỷ trọng số order và subtotal ở mức trung bình
- Nhóm 3: Có tiềm năng, Mặc dù chưa có seller hoặc tập seller và customer còn hạn chế, tuy nhiên đã có tỷ trọng giao dịch có tăng trưởng có thể nhận thấy (không quá chênh lệnh với nhóm 2)

-  Một vài state có rất có tiềm năng để scale up về business như MG,RJ (tỷ trọng số seller thấp, số lượng order và tổng giá trị bán hàng tương đối đáng kể, tỷ trọng subtotal lớn hơn tỷ trọng order có thể suy luận ngay rằng ABV của vùng này lớn hơn trung bình ABV của TB các state)
- Các bang nhóm 3 có thể nghiên cứu thêm về thị trường để có thể phát triển trong tương lai gần

In [43]:
# Đối chiếu thêm thời gian giao vận và tổng thời gian xử lý đơn của các state trên để có cái nhìn rõ hơn về vận hành

time_by_state_2 = time_by_state.loc[time_by_state['customer_state'].isin(list_state_centered)]
time_by_state_plot_2 = plot_bar_distribution_2d(time_by_state_2,'customer_state','type','Time','Thời gian vận chuyển và Tổng thời gian xử lý đơn','Phút','',sort=False,pct=False)

# Kiểm tra về  TB Cost per order theo state
state_cost_per_order = order_state.loc[(order_state['order_status'].isin(['delivered','shipped']))&(order_state['customer_state'].isin(list_state_centered))].\
groupby(['customer_state']).agg({'order_id':pd.Series.nunique,'freight_value':'sum'})
state_cost_per_order['avg_cpo'] = (state_cost_per_order['freight_value']/state_cost_per_order['order_id']).round(decimals=1)
state_cost_per_order= state_cost_per_order.reset_index()
print('Average cost per orser: ',state_cost_per_order['avg_cpo'].mean())
state_cost_per_order_plot = plot_bar_distribution_1d(state_cost_per_order,'customer_state','avg_cpo','TB Cost per order theo state','','','#00587A',sort=False,count=False)
show(column(time_by_state_plot_2,state_cost_per_order_plot))

Average cost per orser:  24.083333333333332


- Có thể nhận thấy các state số lượng nhà cung ứng seller thấp dẫn đến thời gian vận chuyển đơn hàng thường lớn, trong khi các các state có mật độ tập trung seller cao hơn sẽ có thời gian vận chuyển cũng như tổng thời gian xử lý đơn hàng thấp hơn rất nhiều
- Độ phân tán giữa người mua và người bán rộng khiến quãng đường giao vận lớn, dẫn đến Cost per order (chi phí để xử lý 1 đơn hàng) càng lớn
- Vì vậy để tăng trưởng Gross merchandise value và tối ưu về mặt chi phí, việc mở rộng phát triển mạng lưới partner seller tại các state nhóm 2 và nhóm 3 là cần thiết

So sánh thêm về tốc độ tăng trưởng Gross merchandise value theo state

In [44]:
import warnings
warnings.filterwarnings('ignore')

state_growth = order_state.loc[(order_state['order_status'].isin(['delivered','shipped']))&(order_state['customer_state'].isin(list_state_centered))]
state_growth['year'] = pd.to_datetime(state_growth['order_purchase_timestamp']).apply(lambda x: x.year).apply(lambda x: "{:.0f}".format(x))
state_growth_year = state_growth.groupby(['customer_state','year']).agg({'subtotal':sum}).reset_index()
value_distance = state_growth_year['subtotal'].max()*0.1
state_growth_year['distance']= state_growth_year['year'].apply(lambda x: value_distance if x=='2017' else value_distance*2)
state_growth_year['text'] = (state_growth_year.groupby('customer_state')['subtotal']
                                  .apply(pd.Series.pct_change)).fillna(0)
state_growth_year['color']= state_growth_year['year'].apply(lambda x: '#00587A' if x == '2018' else('#FFBD39' if x=='2017' else 'red'))
state_growth_year_plot= plot_bar_distribution_2d(state_growth_year,'customer_state','year','subtotal','Gross merchandise value over year','Gross value','',sort=False, pct=False,text=False)
print('Dữ liệu từ: ',state_growth['order_purchase_timestamp'].min(),'\n',
'Đến: ',state_growth['order_purchase_timestamp'].max())
show(state_growth_year_plot)

Dữ liệu từ:  2016-09-15 12:16:38 
 Đến:  2018-09-03 09:06:57


- Mặc dù dữ liệu mới đến tháng 9/2018, tuy nhiên có thể nhận thấy các state đều có sự tăng trưởng so với năm 2017 mặc dù công ty mới phát triển trong vòng 2 năm (giả sử ngay từ khi hoạt động công ty đã tiến hành thu thập dữ liệu - tức là từ tháng 9 năm 2016)
- SP vẫn là state có sự tăng trưởng cao nhất do mật độ tập trung lượng seller và customer (như đã phân tích ở trên)
- Các state khác có sự tăng trưởng như DF, ES, SC, PR (đây đều là các state có cost per order và thời gian xử lý tương đối thấp so với mặt bằng chung, có sẵn tập khách hàng tuy nhiên lượng seller và customer chưa nhiều, vì vậy nếu scale up business tại các state này có thể maximun về Gross merchandise value và optimize về chi phí)


### Conclusion

Qua việc quá trình exploring data và những phân tích cơ bản phía trên, có thể rút ra được một vài hiểu biết về hoạt động của công ty như sau:
   - Công ty hoạt động trong lĩnh vực TMĐT (có thể là c2c), có tập khách hàng (customers & sellers) tương đối lớn, có phânn bố tại 27 state & 4119 city, tuy nhiên chỉ tập trung chủ yếu tại 1 vài state nhất định.
   - Hơn 40% transaction và Gross merchandise sales tập trung tại 1 bang là SP (nơi tập trung tập khách hàng lớn nhât), tỷ trọng số giao dịch còn lại tập trung rải rác tại các bang khác, các bang có mật độ tập khách hàng cao có số lượng transaction cao hơn => có thể thấy việc phát triển tập khách hàng ảnh hưởng rất lớn đến tăng trưởng số lượng order và gross marchandise sales.
   - Average basket value: Phần lớn các đơn hàng không có giá trị nhỏ và trung bình: 50% số đơn có subtotal < = 86.9; 75% số đơn < = 149.90, nên ABV đơn hàng không lớn. Các sản phẩm được mua bán chủ yếu thuộc nhóm chăm sóc sức khỏe, làm đẹp, đồ gia dụng trong cho nhà cử vì vậy mà giá trị đơn hàng tập trung ở mức nhỏ và trung bình.
   - Payment method: Phần lớn các đơn hàng được thanh toán qua credit card, tuy nhiên các đơn hàng có gia trị lớn hơn mức ABS xu hướng thanh toán qua boleto và sử dụng voucher, hoặc có thể hiểu voucher encourage khách hàng tăng giá trị của giỏ hàng giúp ABS những đơn hàng có voucher lớn hơn ABS toàn bộ các đơn hàng.(dữ liệu này cùng với tập dữ liệu về order item và category có thể được sử dụng để profiling customer). Có thể cung câp thêm nhiều dạng voucher theo mức giá trị giỏ hàng để maximize về average basket size and value.
   - Cost per order (= 24,08): (chi phí vận chuyển)  không tương quan rõ nét với giá trị đơn hàng. Cost per order và tổng thời gian xử lý đơn hàng cao hơn tại các state không có seller, vì vậy có thể suy đoán 1 trong những nguyên nhân dẫn đến cost per order cao là do khoảng cách địa lý, đơn hàng được vận chuyển từ seller ở bang này đến customer ở bang khác có khoảng cách quãng đường xa khiến cost cao.(Có thể đào sâu hơn về kích thước các sản phẩm trong đơn hàng để xác định liệu có phải 1 yếu tố liên quan đến cost per order cao hay không)
   - Service quality: Chất lượng dịch cần thêm nhiều cải thiện do số review cho đánh giá 1 điểm chiếm tỷ trọng tương đối lớn (11.8%)
     -  Thời gian giao hàng và xử lý đơn hàng tương đối lớn (TB > 12 ngày) có thể la nguyên nhân dẫn đến trải nghiệm không tốt cho customer. 
     - Ngoài ra, điểm số này có thể đến từ nhiều lý do như chất lượng sản phẩm, phản hồi/ thái độ của seller, ....(có thể đào sâu hơn để xem số điểm thấp này tập trung ở location nào, seller nào, thời gian xử lý những đơn hàng này bao nhiêu lâu, và trong đơn hàng gồm những category nào để hiểu và tìm solution để improve chất lượng dịch vụ)
   - Dựa vào mật độ tập trung tập khách hàng (customers & sellers), giá trị ABV trung bình theo state, tốc độ tăng trưởng và TB cost per order có thể chia các state thành 4 nhóm:
      - Nhóm 1: (SP) Bước vào giai đoạn phát triển bão hòa (không còn quá nhiều room để tăng trưởng) thể hiện ở số lượng seller và customer vượt trội, tuy nhiên tỷ trọng tập trung seller > tỷ trọng tập trung customer, order > subtotal, ABV cúa state (average basket value) < Trung Bình ABV toàn bộ các order, cost per order và thời gian xử lý đơn thấp
      
     - Nhóm 2:(MG,PR,RJ,RS,SC) Đang phát triển, đã phát triển được 1 tập nhỏ seller và customer, tỷ trọng số order và subtotal ở mức trung bình, cost per order ở mức trung bình, tốc độ tăng trưởng Gross merchandise value tương đối tốt. Một vài state trong nhóm này có ABV > TB ABV cho thấy tiềm năng maximize ABV vẫn còn. Bằng việc tiếp tục phát triển khách hàng (customer lẫn seller) ở các state này có thể tăng được Gross merchandise value và đồng thời có thể giảm được cost per order
     
     - Nhóm 3:(DF,ES, Có tiềm năng, Mặc dù chưa có seller hoặc tập seller và customer còn rất khiêm tốn, tuy nhiên đã có tăng trưởng tỷ trọng giao dịch và Gross merchandise value (không quá chênh lệnh với nhóm 2), 1 vài state còn có cost per order tương tối thấp (thấp hơn nhóm 2) có thể do được hưởng lợi từ vì trí địa lý (gần state có mật độ seller cao), do vậy cũng rất có tiềm năng để khai phá và phát triển tại khu vực này trong tương lai gần
     
     -  Nhóm 4: Các state còn lại
   - Công ty có thể thu thập thêm dữ liệu về quãng đường di chuyển, tracking đầy đủ dữ liệu thời gian giao vận (hiện các mốc thời gian của nhiều đơn hàng vẫn còn bị missing) khi có tập dữ liệu đủ lớn có thể sử dụng thuật toán để tìm ra quãng đường di chuyển ngắn nhất hoặc thuận tiện nhất cho việc gom nhiều đơn hàng và giao các đơn hàng có cùng cung đường
   - Công ty có thể thu thập thêm dữ liệu về nhân khẩu học để profiling đặc điểm khách hàng tại 1 area nhất định, giúp cho việc phát triển thị trường và đưa ra recommendation phù hợp dựa trên khoảng cách seller và customer để giảm thiểu chi phí và thời gian xử lý đơn 
   